# Retrieving Mastodon datasets.

Retrieving Mastodon datasets for specific sentiment analysis.

Written by Luc Bijl.

Retrieving Mastodon key from credentials file.

In [103]:
with open("../.credentials", "r") as file:
    for line in file:
        if 'mastodon-key' in line:
            mastodon_key = line.split('mastodon-key=')[1].strip()
            break

Initializing the Mastodon API.

In [104]:
import pandas as pd
from mastodon import Mastodon

api_base_url = "https://mastodon.social"

mastodon = Mastodon(access_token=mastodon_key,api_base_url=api_base_url)

Queriying all messages containing Eli Lilly in the period 2022-10-28 to 2022-11-25

In [114]:
import pandas as pd
from mastodon import Mastodon

api_base_url = "https://mastodon.social"

mastodon = Mastodon(access_token=mastodon_key,api_base_url=api_base_url)

query = "language:en after:2022-10-28 before:2022-11-25 Eli Lilly"

limit = 1000
max_id = None

date = []
ids = []
content = []

while len(date) < limit:

    toots = mastodon.search(query,result_type='statuses',max_id=max_id)['statuses']

    if not toots:
        break

    if toots[-1].id == max_id:
        break
    
    for n in range(1,len(toots)):
        date.append(toots[n].created_at)
        ids.append(toots[n].id)
        content.append(toots[n].content)

    max_id = toots[-1].id

data = {'Date': date, 'ID': ids, 'Content': content}

df_toots = pd.DataFrame(data)

df_toots

,Date,ID,Content
0,2022-11-19 19:48:27.929000+00:00,109372198612207130,<p>The price was high because Eli Lilly collud...
1,2022-11-19 19:43:11.317000+00:00,109372177858433041,<p>‘More work to do’: Eli Lilly admits <a href...
2,2022-11-19 15:05:00.177000+00:00,109371083991292672,"<p><span class=""h-card"" translate=""no""><a href..."
3,2022-11-19 09:34:21.137000+00:00,109369783819888501,<p>whoa! that was WILD. i completely dissociat...
4,2022-11-18 21:16:12+00:00,109366885198797325,<p>That was the signal?</p><p>Not the loss of ...
5,2022-11-15 19:38:16.330000+00:00,109349509289038689,<p>You Know Who ELSE Cost Eli Lilly Billions O...
6,2022-11-15 01:13:32+00:00,109345166079725205,<p>It's always useful I think to go back and t...
7,2022-11-14 18:38:57.012000+00:00,109343613714853625,<p>“To health care companies such as Eli Lilly...
8,2022-11-14 17:35:19.796000+00:00,109343363549970929,<p>It cost $8 to f*ck over Elon Musk. <br />He...
9,2022-11-14 14:45:47+00:00,109342697052770264,"<p>Free <a href=""https://mstdn.ca/tags/insulin..."
